# 简单等权策略

## 一、等权算法

In [1]:
import os, sys, argparse, logging
import pandas as pd

# 首先将framework的目录加进sys.path中，方便使用相对路径导入
framework_path = os.path.join(os.path.abspath(''), '../framework')
if framework_path not in sys.path:
    sys.path.append(framework_path)

In [2]:
from alg.alg_base import AlgBase

# 等权算法，传入资产字典和总权重
class EqualWeightAlg(AlgBase):
    def __init__(self, name) -> None:
        super().__init__(name)

    def run(self, asset_dict, weight_sum):
        return {asset: weight_sum/len(asset_dict) for asset in asset_dict}

In [3]:
# 测试算法

# 创建一个dummy的数据集
from component.asset.asset import Asset
dummy_dict = {i: Asset(str(i)) for i in range(3)}

ewa = EqualWeightAlg('equal_weight')
print(ewa.run(dummy_dict, 0.6))

init YamlSvc
init DateSvc
{0: 0.19999999999999998, 1: 0.19999999999999998, 2: 0.19999999999999998}


## 二、创建单期策略

In [8]:
from import_func import getSvc
raw_data_svc = getSvc('LxwWinddbRawDataSvc')
date_svc = getSvc('DateSvc')
date_svc.setTradeDays(raw_data_svc.getTradeDays())

from strategy.strategy_base import StrategyBase

class MyStrategy(StrategyBase):
    def __init__(self, name, args={}) -> None:
        super().__init__(name, args={})

    def _initAlgDict(self):
        super()._initAlgDict()
        self._alg_dict['equal_weight'] = EqualWeightAlg('equal_weight')

    def _initDataset(self):
        super()._initDataset()

        a_000001 = Asset('000001.SH')
        a_000001.setRawNavData(raw_data_svc.getNav('aindexeodprices', '000001.SH'))

        a_000003 = Asset('000003.SH')
        a_000003.setRawNavData(raw_data_svc.getNav('aindexeodprices', '000003.SH'))

        self.getDataset().addChildAsset(a_000001)
        self.getDataset().addChildAsset(a_000003)

    def run(self, id_date):
        # first set id_date
        self.getDataset().setIdDateRecursively(id_date, 252)

        alg_input = self.getDataset().getAllAsset(pop_cash=True)
        return self._alg_dict['equal_weight'].run(alg_input, 0.5)


raw_data_svc.sqlQuery: SELECT TRADE_DAYS FROM asharecalendar WHERE S_INFO_EXCHMARKET='SSE' ORDER BY TRADE_DAYS


In [9]:
from datetime import datetime

# 测试单期策略
my_strategy = MyStrategy('test')

my_strategy._dataset.print()
print(my_strategy.run(datetime(2020, 1, 5)))

group: root
	asset: cash
	asset: 000001.SH
	asset: 000003.SH
{'000001.SH': 0.25, '000003.SH': 0.25}
